In [ ]:
from pathlib import Path

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

import locale


In [ ]:
locale.setlocale(locale.LC_TIME, "es_ES.UTF-8")
# pd.options.mode.chained_assignment = None  # default='warn'
plt.style.use("seaborn-notebook")


## Data loading and cleaning

In this section, the applicant loads and cleans raw data from the following files:
- `precipitaciones.csv` has the monthly mean of rainfall registered from January 1979 to April 2020.
- `banco_central.csv` has macroeconomic variables.

In [ ]:
RAINFALL_DATA_PATH = Path("data", "precipitaciones.csv")
MACROECONOMIC_DATA_PATH = Path("data", "banco_central.csv")


### Rainfall _(precipitaciones)_

In [ ]:
import pandas as pd


def load_rainfall_data(data_path):
    df = pd.read_csv(data_path)
    df["date"] = pd.to_datetime(df["date"], format="%Y-%m-%d")
    df = df.sort_values(by="date", ascending=True).reset_index(drop=True)
    df = df.dropna()
    df = df.drop_duplicates()
    return df

precipitaciones = load_rainfall_data(RAINFALL_DATA_PATH)
precipitaciones.head()


### Macroeconomic variables _(banco central)_

In [ ]:
import pandas as pd


def load_macroeconomic_data(data_path):
    df = pd.read_csv(data_path)
    df["Periodo"] = df["Periodo"].apply(lambda x: x[0:10])
    df["Periodo"] = pd.to_datetime(df["Periodo"], format="%Y-%m-%d", errors="coerce")
    df = df.drop_duplicates(subset="Periodo")
    df = df[~df["Periodo"].isna()]
    return df

banco_central = load_macroeconomic_data(MACROECONOMIC_DATA_PATH)
banco_central.head()


In [ ]:
def convert_int(x):
    return int(x.replace('.', ''))

cols_pib = [x for x in list(banco_central.columns) if 'PIB' in x]
cols_pib.extend(['Periodo'])
banco_central_pib = banco_central[cols_pib]
banco_central_pib = banco_central_pib.dropna(how = 'any', axis = 0)

for col in cols_pib:
    if col == 'Periodo':
        continue
    else:
        banco_central_pib[col] = banco_central_pib[col].apply(lambda x: convert_int(x))

banco_central_pib.sort_values(by = 'Periodo', ascending = True)
banco_central_pib

In [ ]:
def to_100(x): #mirando datos del bc, pib existe entre ~85-120 - igual esto es cm (?)
    x = x.split('.')
    if x[0].startswith('1'): #es 100+
        if len(x[0]) >2:
            return float(x[0] + '.' + x[1])
        else:
            x = x[0]+x[1]
            return float(x[0:3] + '.' + x[3:])
    else:
        if len(x[0])>2:
            return float(x[0][0:2] + '.' + x[0][-1])
        else:
            x = x[0] + x[1]
            return float(x[0:2] + '.' + x[2:])

        
cols_imacec = [x for x in list(banco_central.columns) if 'Imacec' in x]
cols_imacec.extend(['Periodo'])
banco_central_imacec = banco_central[cols_imacec]
banco_central_imacec = banco_central_imacec.dropna(how = 'any', axis = 0)

for col in cols_imacec:
    if col == 'Periodo':
        continue
    else:
        banco_central_imacec[col] = banco_central_imacec[col].apply(lambda x: to_100(x))
        assert(banco_central_imacec[col].max()>100)
        assert(banco_central_imacec[col].min()>30)

banco_central_imacec.sort_values(by = 'Periodo', ascending = True)
banco_central_imacec

In [ ]:
banco_central_iv = banco_central[['Indice_de_ventas_comercio_real_no_durables_IVCM', 'Periodo']]
banco_central_iv = banco_central_iv.dropna() # -unidades? #parte 
banco_central_iv = banco_central_iv.sort_values(by = 'Periodo', ascending = True)

In [ ]:
banco_central_iv.head() #unidades? https://si3.bcentral.cl/siete/ES/Siete/Canasta?idCanasta=M57TP1161519 porcentajes?

In [ ]:
banco_central_iv['num'] = banco_central_iv.Indice_de_ventas_comercio_real_no_durables_IVCM.apply(lambda x: to_100(x))
banco_central_iv

In [ ]:
banco_central_iv.Periodo.min()

In [ ]:
banco_central_iv.Periodo.max()

In [ ]:
banco_central_num = pd.merge(banco_central_pib, banco_central_imacec, on = 'Periodo', how = 'inner')
banco_central_num = pd.merge(banco_central_num, banco_central_iv, on = 'Periodo', how = 'inner')

# Supply data and feature engineering.

In this section, the applicant loads the file `precio_leche.csv` which contains the monthly price of milk from 1979 to 2021. She/he/they merges this dataset with the data previously processed and creates new variables to help prediction (feature engineering). Aditionally, some some variables are plotted for exploratory analisys.

In [ ]:
MILK_PRICE_PATH = Path("data", "precio_leche.csv")


In [ ]:
precio_leche = pd.read_csv(MILK_PRICE_PATH)
precio_leche.rename(columns = {'Anio': 'ano', 'Mes': 'mes_pal'}, inplace = True) # precio = nominal, sin iva en clp/litro
precio_leche['mes'] = pd.to_datetime(precio_leche['mes_pal'], format = '%b')
precio_leche['mes'] = precio_leche['mes'].apply(lambda x: x.month)
precio_leche['mes-ano'] = precio_leche.apply(lambda x: f'{x.mes}-{x.ano}', axis = 1)
precio_leche.head()

In [ ]:
precio_leche.plot(x = 'mes-ano', y = 'Precio_leche') #alza 2010-2011?

In [ ]:
precio_leche[precio_leche.ano>=2013].plot(x = 'mes-ano', y = 'Precio_leche')

In [ ]:
precipitaciones['mes'] = precipitaciones.date.apply(lambda x: x.month)
precipitaciones['ano'] = precipitaciones.date.apply(lambda x: x.year)
precio_leche_pp = pd.merge(precio_leche, precipitaciones, on = ['mes', 'ano'], how = 'inner')
precio_leche_pp.drop('date', axis = 1, inplace = True)
precio_leche_pp #precipitaciones fecha_max = 2020-04-01

In [ ]:
banco_central_num['mes'] = banco_central_num['Periodo'].apply(lambda x: x.month)
banco_central_num['ano'] = banco_central_num['Periodo'].apply(lambda x: x.year)
precio_leche_pp_pib = pd.merge(precio_leche_pp, banco_central_num, on = ['mes', 'ano'], how = 'inner')
precio_leche_pp_pib.drop(['Periodo', 'Indice_de_ventas_comercio_real_no_durables_IVCM', 'mes-ano', 'mes_pal'], axis =1, inplace = True)
precio_leche_pp_pib

# Model
In this section, the applicant builds a regression model to predict the price of milk. The model uses and macroeconomic climatological variables loaded at the beginning of this notebook. It also uses features created in the previous section.

In [ ]:
X = precio_leche_pp_pib.drop(['Precio_leche'], axis = 1)
y = precio_leche_pp_pib['Precio_leche']


In [ ]:
import numpy as np
from sklearn.feature_selection import SelectKBest, mutual_info_regression
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import PolynomialFeatures, StandardScaler


K = [3, 4, 5, 6, 7, 10]
ALPHA = [1, 0.5, 0.2, 0.1, 0.05, 0.02, 0.01]
POLY = [1, 2, 3, 5, 7]

def train_model(X, y, verbose=0):
    np.random.seed(0)
    pipeline = Pipeline([
        ("scale", StandardScaler()),
        ("selector", SelectKBest(mutual_info_regression)),
        ("poly", PolynomialFeatures()),
        ("model", Ridge()),
    ])
    grid = GridSearchCV(
        estimator=pipeline,
        param_grid=dict(selector__k=K, poly__degree=POLY, model__alpha=ALPHA),
        cv=3, verbose=verbose, scoring="r2",
    )
    grid.fit(X, y)
    return grid


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

model = train_model(X_train, y_train, verbose=3)
grid = model

y_pred = grid.predict(X_test)
    
# Evaluation
test_rmse = mean_squared_error(y_test, y_pred)
test_r2 = r2_score(y_test, y_pred)
print(f"RMSE={test_rmse}")
print(f"R2={test_r2}")


In [ ]:
grid.best_params_

In [ ]:
X_train.columns[grid.best_estimator_.named_steps['selector'].get_support()]

In [ ]:
predicted = pd.DataFrame(y_test).reset_index(drop = True)
predicted['predicc'] = y_pred
predicted= predicted.reset_index()
plt.scatter(predicted.index, predicted['Precio_leche'], label = 'real')
plt.scatter(predicted.index, predicted['predicc'], color = 'red', label = 'prediccion', alpha = 0.5)
plt.grid(axis = 'x')
plt.legend()

In [ ]:
predicted['residual'] = predicted.Precio_leche - predicted.predicc
plt.hlines(0, xmin = predicted.predicc.min()-10, xmax = predicted.predicc.max()+10, linestyle='--', color = 'black', linewidth = 0.7)
plt.scatter(predicted.predicc, predicted.residual)
plt.xlabel('Predicción')
plt.ylabel('Residuo (y_real - y_pred)')

### Regresión utilizando solamente variables macroeconómicas y climatológicas

In [ ]:
cols_no_leche = [col for col in X.columns if not "leche" in col]
X_train = X_train[cols_no_leche]
X_test = X_test[cols_no_leche]

model = train_model(X_train, y_train, verbose=3)
grid = model

y_pred_noleche = grid.predict(X_test)
    
# Evaluation
test_rmse = mean_squared_error(y_test, y_pred_noleche)
test_r2 = r2_score(y_test, y_pred_noleche)
print(f"RMSE={test_rmse}")
print(f"R2={test_r2}")


In [ ]:
grid.best_params_

In [ ]:
X_train.columns[grid.best_estimator_.named_steps['selector'].get_support()]

In [ ]:
predicted = pd.DataFrame(y_test).reset_index(drop = True)
predicted['predicc'] =y_pred_noleche
predicted= predicted.reset_index()
plt.scatter(predicted.index, predicted['Precio_leche'], label = 'real')
plt.scatter(predicted.index, predicted['predicc'], color = 'red', label = 'prediccion', alpha = 0.5)
plt.grid(axis = 'x')
plt.legend()

In [ ]:
predicted['residual'] = predicted.Precio_leche - predicted.predicc
plt.hlines(0, xmin = predicted.predicc.min()-10, xmax = predicted.predicc.max()+10, linestyle='--', color = 'black', linewidth = 0.7)
plt.scatter(predicted.predicc, predicted.residual)
plt.xlabel('Predicción')
plt.ylabel('Residuo (y_real - y_pred)')